<a href="https://colab.research.google.com/github/GBig-UTP2023/GBig-UTP2023/blob/main/Proyecto_AS_Ver01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!python -m spacy download es_core_news_sm

2023-10-26 23:15:59.744318: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-26 23:15:59.744393: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-26 23:15:59.744428: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-26 23:16:01.080083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 36.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [29]:
###Conjunto de datos de entrenamiento y factorización de los datos

training_data=[
              ("Me encanta el contenido del blog de Analytics Lane, los artículos son fantásticos.", "positivo"),
              ("El código no funciona, me ha dado un error al ejecutarlos.", "negativo"),
              ("Me encanta este producto.","positivo"),
              ("Esta película fue terrible.","negativo"),
              ("El clima está agradable hoy.","positivo"),
              ("Me siento triste por las noticias.","negativo"),
              ("Es solo un libro promedio","neutral")
              ]

In [30]:
## Creando función para analizar conjunto de datos

import spacy

def preprocess_text(text):
  """
  Realiza el preprocesamiento básico de un texto en idioma español utilizando spacy.

  Args:
      txtFrase(str): El texto al ser preprocesado.

  Returns:
      str: El texto preprocesado
 """
  nlp = spacy.load('es_core_news_sm')
  doc = nlp(text)

  # Eliminación de palabra irrelevantes (stopwords) y signos de puntuación

  tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]

  # Recontrucción del text preprocesado
  preprocessed_text=' '.join(tokens)

  return preprocessed_text



In [31]:
## Creando función para crear caracteristicas del texto a analizar.

def extract_features(text):
    """
    Extrae las características del texto utilizando spaCy y devuelve un dicciorario de características.

    Arg:
    txtFrase (str): El texto de cual extraer características.
    Returns:
    dict: Un diccionario que representa las características extraídas del texto.
    """
    features={}
    doc = nlp (text)
    for token in doc:
      if not token.is_stop and not token.is_punct:
        if token.lemma_.lower() in features:
          features[token.lemma_.lower()] += 1
        else:
          features[token.lemma_.lower()] = 1
    return features





In [32]:
#Entrenamiento del modelo ::::En AS el modelo que mejor funciona es Naive Bayes spacy no cuenta con una implementación propia,
#                             pero es posible usar Scikit-learn que consta de preprocesar los datos, extraer las caracteristicas y crear un conjunto de entrenamiento
#                             para el modelo MultinominalNB()

from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB

#Preprocesamiento de los datos de entrenamiento

preprocessed_training_data=[(preprocess_text(text), label) for text, label in training_data]

#Extracción de características de los datos de entrenamiento

training_features = [extract_features(text) for text, _ in
                     preprocessed_training_data]
vectorizer = DictVectorizer(sparse=False)
X_train = vectorizer.fit_transform(training_features)

#Etiquetas de los datos de entrenamiento

y_train = [label for _, label in preprocessed_training_data]

#Entrenamiento del clasificador Naive Bayer

classifier = MultinomialNB()
_ = classifier.fit(X_train, y_train)

In [36]:
#Nuevo texto para clasificar
new_text="Es un carro expetacular"

#Preprocesamiento del nuevo texto
preprocessed_text = preprocess_text(new_text)

#Extracción de características del nuevo texto

features=extract_features(preprocessed_text)
X_test=vectorizer.transform([features])

#Clasificación del nuevo texto
sentiment = classifier.predict(X_test)
print("Sentimiento: ", sentiment[0])

Sentimiento:  negativo
